# Downloading Cube Example and Decompression

For users **not** using public HETDEX JupyterHub access, cubes may be downloaded on demand. This is an example to download 1 datacube. This cube is used in notebooks 03-05. Please see notebook 09-BatchDownloads+Decompression.ipynb to download a large number of cubes. If you are on our JupyterHub, you can skip this notebook if you like!

In [1]:
import os.path as op
from astropy.io import fits

## Example Download Script for a single IFU cube

In [2]:
# Base URL
base_url = "http://web.corral.tacc.utexas.edu/hetdex/HETDEX/internal/pdr1/datacubes"
shotid = 20190405020
ifuslot = '034'

filename = f"dex_cube_{shotid}_{ifuslot}.fits"
url = f"{base_url}/{shotid}/{filename}"

print('The datacube is found at:')
print(url)

The datacube is found at:
http://web.corral.tacc.utexas.edu/hetdex/HETDEX/internal/pdr1/datacubes/20190405020/dex_cube_20190405020_034.fits


Now that you have the url list you can either download in a notebook. Or if you prefer opening a ssh/terminal window, you can use wget. Example wget command to download the files, preserving the pdr1 data file structure so all notebooks will work provided you point to the top level directory where this command is performed. Ideally same directory as ifu-index.fits

    wget --user='hetdex_internal' --ask-password --cut-dirs=4 -nH -x -i urls.txt

If you would like to download in parallel you can use this unix function, see Notebook 09 for more examples.

    cat urls.txt | xargs -n 1 -P 4 wget --user='hetdex_internal' --password='your_password' --cut-dirs=4 -nH -x

## Or if you prefer you can use this code to download within this JupyterNotebook.

In [3]:
import os
import getpass
import subprocess
from urllib.parse import urlsplit

username = 'hetdex_internal'
password = getpass.getpass('Password: ')

# this is where you want to store the downloaded cubes. This will retain the directory structure of the data model
pdr_dir = '/home/jovyan/work/'

base_url_path = '/hetdex/HETDEX/internal/pdr1/'  # everything before this will be cut

path = urlsplit(url).path
rel_path = path.split(base_url_path, 1)[-1]  # Extract relative path under pdr1/
local_path = os.path.join(pdr_dir, rel_path)

# Ensure directory exists
os.makedirs(os.path.dirname(local_path), exist_ok=True)

print(f"Downloading: {rel_path}")
subprocess.run([
    'wget', '-q',
    '--user', username,
    '--password', password,
    '-O', local_path,
    url
])

Password:  ········


Downloading: datacubes/20190405020/dex_cube_20190405020_034.fits


CompletedProcess(args=['wget', '-q', '--user', 'hetdex_internal', '--password', 'lae1216mcdonald', '-O', '/home/jovyan/work/datacubes/20190405020/dex_cube_20190405020_034.fits', 'http://web.corral.tacc.utexas.edu/hetdex/HETDEX/internal/pdr1/datacubes/20190405020/dex_cube_20190405020_034.fits'], returncode=0)

### We suggest you decompress the files if you don't mind increasing the size by 3 times. It will make read times much faster. 

In [4]:
#function to decompress the RICE-compressed IFU Fits cubes

def decompress_ifu(shotid, ifuslot, pdr_dir, verbose=False):
    """
    Decompress a RICE-compressed IFU FITS cube and overwrite the original file.
    Returns a tuple: (shotid, ifuslot, output_path, success_flag, error_message)
    """
    infile = op.join(pdr_dir, 'datacubes', str(shotid), f'dex_cube_{shotid}_{ifuslot}.fits')

    if not op.exists(infile):
        return (shotid, ifuslot, None, False, f"Input file not found: {infile}")

    try:
        with fits.open(infile, memmap=False) as hdul:
            new_hdul = fits.HDUList()
            for hdu in hdul:
                if isinstance(hdu, fits.CompImageHDU):
                    new_hdul.append(fits.ImageHDU(data=hdu.data, header=hdu.header))
                else:
                    new_hdul.append(hdu.copy())
            new_hdul.writeto(infile, overwrite=True)

        if verbose:
            print(f"✓ Decompressed in place: {shotid}-{ifuslot}")
        return (shotid, ifuslot, infile, True, None)

    except Exception as e:
        err_msg = traceback.format_exc()
        if verbose:
            print(f"✗ Failed: {shotid}-{ifuslot}\n{err_msg}")
        return (shotid, ifuslot, None, False, err_msg)

In [5]:
decompress_ifu(shotid, ifuslot, pdr_dir, verbose=True)


✓ Decompressed in place: 20190405020-034


(20190405020,
 '034',
 '/home/jovyan/work/datacubes/20190405020/dex_cube_20190405020_034.fits',
 True,
 None)